**NOTA**: Si detectas algún error en este Colab, alguna particularidad que haya que instalar, etc. Pon un mensaje en el foro para que lo podamos solucionar o envía un correo.

# 1 Entorno de trabajo para las prácticas

En esta asignatura vamos a trabajar con una máquina **Ubuntu 20.04**. Para ello, tienes dos opciones. La **primera** es utilizar una máquina de Polilabs que encontrarás disponible en la siguiente dirección:
https://polilabs.upv.es/

Entrando con tu usuario y contraseña, deberás hacer clic en la máquina **EPSG Linux**, la cual te abrirá ya el sistema operativo. Ten en cuenta que los cambios que realices en esta máquina no se guardarán de una sesión a otra, por lo que todo lo que quieras que se guarde, lo deberás guardar en `$HOME/W`. 

La **segunda** opción es crear una máquina virtual en tu ordenador, bien instalada nativamente o bien utilizando un software de virtualización (como Virtualbox o VMWare). Aunque la mayoría del trabajo lo puedes realizar en Polilabs, se **recomienda al 100%** tener también una máquina local. Entre otras cosas, la máquina local será necesaria para trabajar con el robot físico.

En las siguientes secciones se explica los pasos para instalar todo lo necesario en una máquina local. Estos pasos **NO** deberías hacerlos en la máquina de Polilabs.

# 2 Preparación de la máquina local (virtual)

En nuestro caso partiremos de una máquina virtual en Virtualbox, aunque podrías utilizar otro software e incluso instalarla nativamente. En concreto, disponemos de unos requisitos de **2048 MB** de memoria RAM y un disco de **25 GB**. Le ponemos también 1 CPU aunque si es posible, mejor 2.

A continuación, descargamos un Ubuntu 20.04 desktop y lo instalamos en esta máquina: https://ubuntu.com/download/desktop

Ten en cuenta que cuando arrancas la máquina virtual por primera vez, deberás especificar dónde está la ISO para arrancar la instalación.

Una vez terminada la instalación, aunque tiene ya pantalla completa, instalamos también **Guest additions** para que nos deje hacer portapapeles bidireccional. Para ello, una vez logueados dentro de Ubuntu hacemos clic en el menú superior de Virtualbox **Dispositivos** y después en **Insertar imagen de CD de las Guest Additions**. 

Siguiendo las instrucciones y una vez reiniciada la máquina, podemos hacer clic en **Dispositivos > Portapapeles compartido > Bidireccional**. Para poder copiar y pegar código entre la máquina host y la virtual.

**NOTA**: si la pantalla inicial se queda negra, hay que minimizar a pantalla pequeña y cuando entra en escritorio, ya funciona correctamente. 

Instalamos git y **opcionalmente** el editor emacs (aunque puedes utilizar otro como nano, vi, etc.):

In [ ]:
sudo apt install git
sudo apt install emacs

# 3 Añadir repositorio de ROS2

Ahora vamos a instalar ROS2. El primer paso es añadir el repositorio de ROS2:

In [ ]:
#autorizar la clave GPG de ROS
sudo apt update && sudo apt install curl gnupg2 lsb-release
sudo curl -sSL https://raw.githubusercontent.com/ros/rosdistro/master/ros.key  -o /usr/share/keyrings/ros-archive-keyring.gpg

In [ ]:
#añadir el repositorio
echo "deb [arch=$(dpkg --print-architecture) signed-by=/usr/share/keyrings/ros-archive-keyring.gpg] http://packages.ros.org/ros2/ubuntu $(source /etc/os-release && echo $UBUNTU_CODENAME) main" | sudo tee /etc/apt/sources.list.d/ros2.list > /dev/null

# 4 Instalación basada en paquetes

En primer lugar, actualizamos la cache de repositorios e instalamos el desktop **Foxy** (recomendado). Instala ROS, RViz, demos y tutoriales:

In [ ]:
sudo apt update 
sudo apt install ros-foxy-desktop

Esto instala más de 900 paquetes. Como **alternativa** se puede instalar solo la base, pero no hay herramientas GUI. Instala bibliotecas de comunicación, paquetes de mensajes y órdenes de línea de comandos:

`sudo apt install ros-foxy-ros-base`

Instalamos también el compilador **colcon**:

In [ ]:
sudo apt-get install python3-colcon-common-extensions

# 5 Configuración del entorno

In [ ]:
source /opt/ros/foxy/setup.bash

**Opcionalmente**, podemos instalar `argcomplete` para autocompletar desde la línea de comandos:
`sudo apt install -y python3-argcomplete`

# 6 Ejemplos para probar

A continuación ejecutaremos un productor que crearán mensajes y un consumidor que leerá los mensajes emitidos:

In [ ]:
#terminal 1
ros2 run demo_nodes_cpp talker

#terminal 2
source /opt/ros/foxy/setup.bash
ros2 run demo_nodes_py listener

Debería verse que el **talker** indica que está publicando mensajes y al **listener** que los está escuchando. De esta manera, se prueban las API C++ y Python al mismo tiempo.

# 7 Siguientes pasos

Tras instalar ROS2,  hay que incluir los paquetes que se detallan en
https://emanual.robotis.com/docs/en/platform/turtlebot3/quick-start/

En esta URL también explica cómo instalar Ubuntu y cómo instalar ROS2 sobre Ubuntu. **Importante**: Hay que seleccionar **Foxy** en la parte superior de la página.

A continuación, se resumen los paquetes a instalar:

In [ ]:
#gazebo 11
sudo apt-get install ros-foxy-gazebo-*

#cartographer 
sudo apt install ros-foxy-cartographer ros-foxy-cartographer-ros

#navigation 2
sudo apt install ros-foxy-navigation2 ros-foxy-nav2-bringup

#turtlebot 3
source ~/.bashrc
sudo apt install ros-foxy-dynamixel-sdk ros-foxy-turtlebot3-msgs ros-foxy-turtlebot3

#configuración del entorno
echo 'export ROS_DOMAIN_ID=30 #TURTLEBOT3' >> ~/.bashrc
source ~/.bashrc

Para facilitar las cosas al usuario y que no tenga que configurar la variable **ROS_DOMAIN** cada vez que se conecte al sistema (lo configurado en ~/.bashrc se pierde cuando se restaure una máquina virtual) se crea un fichero `/usr/local/bin/foxy` para que el usuario ejecute simplemente `source foxy` con el siguiente contenido:

In [ ]:
source /opt/ros/foxy/setup.sh
export ROS_DOMAIN_ID=30 #TURTLEBOT3
source /usr/share/gazebo/setup.sh

Y le damos permiso de ejecución para todos:

In [ ]:
sudo chmod +x /usr/local/bin/foxy

Ahora vamos a probar que todo funciona correctamente:

In [ ]:
#terminal 1
source foxy
gazebo --verbose /opt/ros/foxy/share/gazebo_plugins/worlds/gazebo_ros_diff_drive_demo.world
#en este punto aparecerá gazebo con un mundo y un vehículo

#terminal 2
source foxy
ros2 topic pub /demo/cmd_demo geometry_msgs/Twist '{linear: {x: 1.0}}' -1
#el vehículo empezará a moverse